In [2]:
import os
from typing import Dict, List
from tavily import TavilyClient
import google.generativeai as genai
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv 

In [4]:
# Load environment variables from .env file
load_dotenv()

True

In [6]:
# Configure Google's Generative AI with the API key from environment variables
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [8]:
# Initialize the Gemini 1.5 Pro model from Google's Generative AI
llm = genai.GenerativeModel('gemini-1.5-pro')

In [10]:
# Initialize TavilyClient with the API key from environment variables
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

In [12]:
# Define the research agent function
def research_agent(state: Dict):
    # Perform a search using Tavily with the query from the state
    response = tavily.search(state["query"])
    # Return the search results along with the existing state
    return {
        "research_results": response.get('results', []),
        "drafted_answer": state["drafted_answer"],
        "query": state["query"]
    }

In [14]:
# Define the drafting agent function
def drafting_agent(state: Dict):
    # Combine the content from all research results into a single context string
    context = "\n".join([res.get('content', '') for res in state["research_results"]])
    # Generate a summary of the context using the Gemini model
    response = llm.generate_content(f"Summarize: {context}")
    # Return the research results, the drafted answer, and the query
    return {
        "research_results": state["research_results"],
        "drafted_answer": response.text,
        "query": state["query"]
    }

In [16]:
# Define the function to build the workflow
def build_workflow():
    workflow = StateGraph(dict)
    workflow.add_node("research", research_agent)
    workflow.add_node("drafting", drafting_agent)
    workflow.add_edge("research", "drafting")
    workflow.add_edge("drafting", END)
    workflow.set_entry_point("research")
    return workflow.compile()

In [18]:
# Define the function to run the research system
def run_research_system(query: str):
    # Build the workflow
    workflow = build_workflow()
    # Invoke the workflow with the initial state
    result = workflow.invoke({
        "research_results": [],
        "drafted_answer": "",
        "query": query
    })
    # Return the drafted answer from the workflow result
    return result["drafted_answer"]

In [20]:
# Main entry point of the script
if __name__ == "__main__":
    # Run the research system with a sample query
    response = run_research_system("Impact of AI on cybersecurity in points")
    # Print the response
    print(response)

AI is significantly impacting cybersecurity, with CISOs recognizing its influence and organizations increasingly relying on it for defense.  The year 2025 is predicted to be a pivotal year for AI in cybersecurity, bringing both opportunities and challenges.  AI is transforming cybersecurity operations, particularly in threat detection and incident response, but its integration also necessitates new security measures.  Cybersecurity providers who understand, embrace, and offer AI-powered solutions, including securing AI implementations themselves, will have a competitive edge.  This includes expanding AI's role in cybersecurity beyond SecOps to areas like cloud and endpoint security.  Nationally and internationally, managing the cybersecurity implications of AI is crucial for cyber-resilience as AI becomes more integrated into critical infrastructure and society.

